In [20]:
from datasets import load_dataset,Image
dataset = load_dataset("csv", data_files=r"C:\Users\User\Projects\FYP2\Training_Sample_5pct\stage2_train_metadata.csv")

In [21]:
for k,v in dataset.items():
    print(k)
    print(v)
    break

train
Dataset({
    features: ['patientId', 'x', 'y', 'width', 'height', 'Target', 'class', 'age', 'sex', 'modality', 'position'],
    num_rows: 1500
})


In [27]:
import pandas as pd
from datasets import Dataset, Image
from flwr_datasets.partitioner import Partitioner
import os

# Step 1: Load your CSV
df = pd.read_csv("metadata.csv", names=[
    "patient_id", "x", "y", "width", "height", "target", 
    "diagnosis", "age", "sex", "modality", "view"
])

# Step 2: Create full image paths from patient IDs
# Assuming images are named like: {patient_id}.png or {patient_id}.jpg
image_dir = r"C:\Users\User\Projects\FYP2\Training_Sample_5pct\Images"
df["image_path"] = df["patient_id"].apply(
    lambda x: os.path.join(image_dir, f"{x}.png")  # or .jpg depending on your format
)

# Step 3: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 4: Cast the image_path column to Image type
# This tells HuggingFace to automatically load and decode images as PIL Images
dataset = dataset.cast_column("image_path", Image())

# Now your dataset is ready!
print(dataset[0])
# Output will show: {'patient_id': '...', 'target': 1, 'image_path': <PIL Image>, ...}


FileNotFoundError: [Errno 2] No such file or directory: 'metadata.csv'

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['patientId', 'x', 'y', 'width', 'height', 'Target', 'class', 'age', 'sex', 'modality', 'position'],
        num_rows: 1500
    })
})

In [24]:
from datasets import load_dataset, Image
from flwr_datasets.partitioner import IidPartitioner 

In [25]:
partitioner = IidPartitioner(num_partitions=10)
partitioner.dataset = dataset["train"]  # Use the appropriate split
partition = partitioner.load_partition(partition_id=0)

In [ ]:
import pandas as pd
from datasets import Dataset, Image
from flwr_datasets.partitioner import IidPartitioner
import os
from sklearn.model_selection import train_test_split

# Step 1: Load your CSV
df = pd.read_csv(r"C:\Users\User\Projects\FYP2\Training_Sample_5pct\stage2_train_metadata.csv")

# Step 2: Create full image paths from patient IDs
# Assuming images are named like: {patient_id}.png or {patient_id}.jpg
image_dir = r"C:\Users\User\Projects\FYP2\Training_Sample_5pct\Images"
df["image_path"] = df["patientId"].apply(
    lambda x: os.path.join(image_dir, f"{x}.png")  # or .jpg depending on your format
)

# Step 3: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 4: Cast the image_path column to Image type
# This tells HuggingFace to automatically load and decode images as PIL Images
dataset = dataset.cast_column("image_path", Image())

train_dataset, test_dataset = dataset.train_test_split(test_size=0.2,seed=42)

# Now your dataset is ready!|
print(train_dataset[0])
# Output will show: {'patient_id': '...', 'target': 1, 'image_path': <PIL Image>, ...}
partioner = IidPartitioner(num_partitions=10)
partion  = dataset.load_partition(partition_id=0)


t


In [37]:
import numpy as np
class DiseaseBasedPartitioner(Partitioner):
    """Partition by disease type for non-IID federated learning."""
    
    def __init__(self, num_partitions: int, target_column: str = "target"):
        super().__init__()
        self._num_partitions = num_partitions
        self._target_column = target_column
        self._partition_id_to_indices = {}
    
    def load_partition(self, partition_id: int):
        self._determine_partition_id_to_indices_if_needed()
        return self.dataset.select(self._partition_id_to_indices[partition_id])
    
    def _determine_partition_id_to_indices_if_needed(self):
        if self._partition_id_to_indices:
            return
        
        # Get target labels
        targets = np.array(self.dataset[self._target_column])
        
        # Get indices for positive and negative cases
        positive_indices = np.where(targets == 1)[0]
        negative_indices = np.where(targets == 0)[0]
        
        # Shuffle
        np.random.seed(42)
        np.random.shuffle(positive_indices)
        np.random.shuffle(negative_indices)
        
        # Distribute with different ratios per client (simulating hospital variations)
        for partition_id in range(self._num_partitions):
            # Vary the positive/negative ratio per client
            ratio = 0.3 + (partition_id * 0.1)  # Different prevalence per client
            
            n_positive = len(positive_indices) // self._num_partitions
            n_negative = int(n_positive * (1 - ratio) / ratio)
            
            start_pos = partition_id * n_positive
            start_neg = partition_id * n_negative
            
            partition_indices = np.concatenate([
                positive_indices[start_pos:start_pos + n_positive],
                negative_indices[start_neg:start_neg + n_negative]
            ]).tolist()
            
            self._partition_id_to_indices[partition_id] = partition_indices
    
    @property
    def num_partitions(self) -> int:
        return self._num_partitions


NameError: name 'tokenizer' is not defined

In [1]:
from federated_pneumonia_detection.src.utils.config_loader import ConfigLoader
config_loader = ConfigLoader(
        config_dir=r"federated_pneumonia_detection\config",
        
    )
config = config_loader.load_config()
    

ModuleNotFoundError: No module named 'federated_pneumonia_detection'